# 16S_procedure_3b_bdiv_australia_analysis: Here we will test the hypothesis that compartments have distinct microbial communities via beta-diversity assessment

# NOTE: These anlyses utilized rep_set.tre, which was VERY poor. All resulting analyses have been move to: output/old_tree_analyses

# For analysis with updated tree (constrained to Green Genes phylogeny), see: 16S_procedure_3b_bdiv_australia_analysis_gg_constrained_tree

In [1]:
cd ..

/Users/FJPollock/Dropbox/coral_microbiomes_postdoc/gcmp_osu_psu_shared_folder/coral_microbe_coevolution/Projects/australia_coevolution_paper/16S_analysis/3b_bdiv_australia_analysis


In [2]:
#Import libraries
from os.path import join,abspath
from os import listdir

In [4]:
#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map.txt")
output_dir = abspath("output/")

#Preference file indicating that Curtis-Brays distance should also be calculated by beta_diversity
bc_prefs_fp = abspath("input/bdiv_bc_prefs.txt")

In [5]:
#Subsampling to 1000 loses 31 samples, but this seems reasonable.
depth=1000

In [ ]:
#Run beta diversity on all samples
bdiv_dir_all = join(output_dir,'bdiv_all')
!beta_diversity_through_plots.py -i $otu_table -o $bdiv_dir_all -m $mapping -p $bc_prefs_fp --force -t $tree_fp -e $depth

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.200167655029 and the largest is 17.3868973942.
  RuntimeWarning


In [45]:
#Export Sceleractinian coral compartment specific biom files (i.e. coral t, m and s)
subset_output = join(output_dir,'tissue_compartment_tables')
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_mucus.biom -m $mapping -s 'sample_type:Coral_Mucus'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_skeleton.biom -m $mapping -s 'sample_type:Coral_Skeleton'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_tissue.biom -m $mapping -s 'sample_type:Coral_Tissue'

In [ ]:
#Summarize BIOM to confirm filter functioned correctly
!biom summarize-table -i $subset_output/otu_table_subset_mucus.biom -o $subset_output/otu_table_subset_mucus_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_tissue.biom -o $subset_output/otu_table_subset_tissue_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_skeleton.biom -o $subset_output/otu_table_subset_skeleton_summary.txt

#Summaries look good.

In [49]:
#Run beta diversity on rarified (1000) subset of each sample type
tissue_otu_table_dir = join(output_dir,"tissue_compartment_tables")
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.103036605043 and the largest is 9.90013989806.
  RuntimeWarning
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the

In [50]:
#Check for differences among mapping categories within a given compartment
tissue_otu_table_dir = join(output_dir,"tissue_compartment_tables")

n_permutations = 1000
categories = 'field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(output_dir,'bdiv_%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

['bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'log_20160426110646.txt', 'otu_table_subset_mucus_even1000.biom', 'unweighted_unifrac_dm.txt', 'unweighted_unifrac_emperor_pcoa_plot', 'unweighted_unifrac_pc.txt', 'weighted_unifrac_dm.txt', 'weighted_unifrac_emperor_pcoa_plot', 'weighted_unifrac_pc.txt']
curr_cat: field_host_clade_sensu_fukami
curr_metric: bray_curtis
curr_category: otu_table_subset_mucus.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                 Df SumsOfSqs MeanSqs F.Model     R2   Pr(>F)
qiime.data$map[[opts$category]]  14     9.757 0.69696  1.6898 0.1077 0.000999
Residuals                       196    80.840 0.41245         0.8923         
Total                           210    90.598                 1.0000         
                        

In [7]:
#Boxplot summaries: pre
tissue_otu_table_dir = join(output_dir,"tissue_compartment_tables")
n_permutations = 1000
categories = 'field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id,photosynthetically_active_radiation,visibility,turf_contact_percent,n_macroalgal_contacts,cyanobacteria_percent,cyanobacteria_contact,binary_turf_contact,binary_macroalgal_contact'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(output_dir,'bdiv_%s' %c)
    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

['bray_curtis_dm.txt', 'bray_curtis_emperor_pcoa_plot', 'bray_curtis_pc.txt', 'compare_categories_field_host_clade_sensu_fukami_bray_curtis', 'compare_categories_field_host_clade_sensu_fukami_unweighted_unifrac', 'compare_categories_field_host_clade_sensu_fukami_weighted_unifrac', 'compare_categories_field_host_genus_id_bray_curtis', 'compare_categories_field_host_genus_id_unweighted_unifrac', 'compare_categories_field_host_genus_id_weighted_unifrac', 'compare_categories_field_host_name_bray_curtis', 'compare_categories_field_host_name_unweighted_unifrac', 'compare_categories_field_host_name_weighted_unifrac', 'compare_categories_reef_name.x_bray_curtis', 'compare_categories_reef_name.x_unweighted_unifrac', 'compare_categories_reef_name.x_weighted_unifrac', 'distance_boxplots_all_within_bray_curtis', 'distance_boxplots_all_within_unweighted_unifrac', 'distance_boxplots_all_within_weighted_unifrac', 'log_20160426110646.txt', 'otu_table_subset_mucus_even1000.biom', 'unweighted_unifrac_dm

In [54]:
#Test for differences in all samples (i.e. not partioned by compartment). This is useful for identifying differences among compartments.
categories = 'BiologicalMatter,tissue_compartment,field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id,photosynthetically_active_radiation,visibility,turf_contact_percent,n_macroalgal_contacts,cyanobacteria_percent,cyanobacteria_contact,binary_turf_contact,binary_macroalgal_contact'
for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
    curr_boxplot_output_dir = join(bdiv_dir_all,"distance_boxplots_%s" %metric)
    curr_dm = join(bdiv_dir_all,"%s_dm.txt" %metric)
    !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median


/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [12]:
#Run beta diversity on all samples in dataset rarified to 1000 reads
tissue_otu_table_dir = join(output_dir,"bdiv_all")
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(output_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)

!beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.211066508956 and the largest is 17.3499799071.
  RuntimeWarning
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the

In [10]:
#Convert unrarified BIOM to CSV for manipulation in PRIMER
!biom convert -i $input_dir/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom -o $input_dir/otu_table_mc2_wtax_no_pynast_failures_no_organelles_TXT.txt --to-tsv --header-key taxonomy

In [13]:
#Convert rarified (1000) BIOM to CSV for manipulation in PRIMER
!biom convert -i $output_dir/bdiv_otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000.biom/otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000.biom -o $input_dir/otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000_TXT.txt --to-tsv --header-key taxonomy

# Test for significant differences in community composition among coral compartments

In [24]:
#Test for significant differences among coral compartments
!compare_categories.py --method adonis -i output/bdiv_all/unweighted_unifrac_dm.txt -m input/gcmp16S_map_r20.txt -c BiologicalMatter -o output/significance_testing_compartments/compare_categories_compartments_unweighted_unifrac_adonis.txt -n 999

In [11]:
#Explore which OTUs differ significantly among compartments
!group_significance.py -i input/otu_table_mc2_wtax_no_pynast_failures_no_organelles_coral_tissue_mucus_skeleton_only.biom -m input/gcmp16S_map_r20.txt -c BiologicalMatter -s g_test -o output/significance_testing_compartments/group_significance_OTU_BiologicalMatter_g_test.txt

/macqiime/anaconda/lib/python2.7/site-packages/qiime/stats.py:1247: RuntimeWarning: divide by zero encountered in double_scalars
  q = 1. + (a + 1.) / (6. * n)


# Does host genus drives microbiome structure in brooders to a greater extent than spawners? We will also look at reef and season.

In [3]:
#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map.txt")
output_dir = abspath("output/")

#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map_r20.txt")
output_dir = abspath("output/")

In [9]:
#Split OTU table into brooder- and spawner-specific biom files
subset_output = join(output_dir,'brooder_v_spawner')
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_brooder.biom -m $mapping -s 'Mode_of_larval_development:brooder'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_spawner.biom -m $mapping -s 'Mode_of_larval_development:spawner'

In [10]:
#Export Sceleractinian coral compartment specific biom files (i.e. coral t, m and s) for brooders
subset_output = join(output_dir,'brooder_v_spawner')
brooder_otu_table = join(subset_output,'otu_table_subset_brooder.biom')
spawner_otu_table = join(subset_output,'otu_table_subset_spawner.biom')
#Export Sceleractinian coral compartment specific biom files (i.e. coral t, m and s) for brooders
!filter_samples_from_otu_table.py -i $brooder_otu_table -o $subset_output/otu_table_subset_brooder_mucus.biom -m $mapping -s 'BiologicalMatter:Coral Mucus'
!filter_samples_from_otu_table.py -i $brooder_otu_table -o $subset_output/otu_table_subset_brooder_skeleton.biom -m $mapping -s 'BiologicalMatter:Coral Skeleton'
!filter_samples_from_otu_table.py -i $brooder_otu_table -o $subset_output/otu_table_subset_brooder_tissue.biom -m $mapping -s 'BiologicalMatter:Coral Tissue'
#Export Sceleractinian coral compartment specific biom files (i.e. coral t, m and s) for spawners
!filter_samples_from_otu_table.py -i $spawner_otu_table -o $subset_output/otu_table_subset_spawner_mucus.biom -m $mapping -s 'BiologicalMatter:Coral Mucus'
!filter_samples_from_otu_table.py -i $spawner_otu_table -o $subset_output/otu_table_subset_spawner_skeleton.biom -m $mapping -s 'BiologicalMatter:Coral Skeleton'
!filter_samples_from_otu_table.py -i $spawner_otu_table -o $subset_output/otu_table_subset_spawner_tissue.biom -m $mapping -s 'BiologicalMatter:Coral Tissue'

In [13]:
#Summarize BIOM to confirm filter functioned correctly
subset_output = join(output_dir,'brooder_v_spawner')
!biom summarize-table -i $subset_output/otu_table_subset_brooder_mucus.biom -o $subset_output/otu_table_subset_brooder_mucus_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_brooder_skeleton.biom -o $subset_output/otu_table_subset_brooder_skeleton_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_brooder_tissue.biom -o $subset_output/otu_table_subset_brooder_tissue_summary.txt

!biom summarize-table -i $subset_output/otu_table_subset_spawner_mucus.biom -o $subset_output/otu_table_subset_spawner_mucus_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_spawner_skeleton.biom -o $subset_output/otu_table_subset_spawner_skeleton_summary.txt
!biom summarize-table -i $subset_output/otu_table_subset_spawner_tissue.biom -o $subset_output/otu_table_subset_spawner_tissue_summary.txt

#Looks good!

In [5]:
#Run beta diversity on rarified (1000) subset of each sample type
tissue_otu_table_dir = join(output_dir,"brooder_v_spawner")
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)
    print(curr_otu_table)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/Users/FJPollock/Dropbox/coral_microbiomes_postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/brooder_v_spawner/otu_table_subset_brooder.biom
/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0412282144692 and the largest is 3

In [8]:
#Check for differences among mapping categories within a given compartment
tissue_otu_table_dir = join(output_dir,"brooder_v_spawner")

n_permutations = 1000
categories = 'field_host_clade_sensu_fukami,reef_name.x,field_host_name,field_host_genus_id'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
 #   print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

curr_cat: field_host_clade_sensu_fukami
curr_metric: bray_curtis
curr_category: bdiv_otu_table_subset_brooder.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                Df SumsOfSqs MeanSqs F.Model      R2   Pr(>F)
qiime.data$map[[opts$category]]  1    2.2918 2.29178  5.6962 0.09873 0.000999
Residuals                       52   20.9213 0.40233         0.90127         
Total                           53   23.2131                 1.00000         
                                   
qiime.data$map[[opts$category]] ***
Residuals                          
Total                              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
curr_cat: reef_name.x
curr_metric: bray_curtis
curr_category: bdiv_otu_table_subset_brooder.biom

Call:
adonis(formula = as.dist(qiime.data$

# We see significant correlations between alpha diversity and 1) latitude and 2) proportion of max colony size. These effects are most pronounced in stress-tolerant corals. Here, we will see if overall community structure changes 1) along latitudinal gradient and 2) as corals grow larger. 

In [4]:
#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map_r20.txt")
output_dir = abspath("output/")

In [4]:
#Split OTU table into different functional groups
subset_output = join(output_dir,'functional_groups')
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_generalist.biom -m $mapping -s 'functional_group_sensu_darling:Generalist'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_competitive.biom -m $mapping -s 'functional_group_sensu_darling:Competitive'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_stress_tolerant.biom -m $mapping -s 'functional_group_sensu_darling:Stress-tolerant'
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_subset_weedy.biom -m $mapping -s 'functional_group_sensu_darling:Weedy'


In [5]:
#Summarize BIOM to confirm filter functioned correctly
subset_output = join(output_dir,'functional_groups')
!biom summarize-table -i $subset_output/otu_table_subset_generalist.biom -o $subset_output/otu_table_subset_generalist.txt
!biom summarize-table -i $subset_output/otu_table_subset_competitive.biom -o $subset_output/otu_table_subset_competitive.txt
!biom summarize-table -i $subset_output/otu_t able_subset_stress_tolerant.biom -o $subset_output/otu_table_subset_stress_tolerant.txt
!biom summarize-table -i $subset_output/otu_table_subset_weedy.biom -o $subset_output/otu_table_subset_weedy.txt


#Looks good!

In [5]:
#Run beta diversity on rarified (1000) subset of each sample type
tissue_otu_table_dir = join(output_dir,'functional_groups')
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)
    print(curr_otu_table)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3b_bdiv_australia_analysis/output/functional_groups/otu_table_subset_competitive.biom
/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.00741759659045 and the l

In [7]:
#Check for differences among mapping categories within a given functional group
tissue_otu_table_dir = join(output_dir,"functional_groups")

n_permutations = 1000
categories = 'prop_Colony_maximum_diameter_universal,latitude,longitude'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

curr_cat: prop_Colony_maximum_diameter_universal
curr_metric: bray_curtis
curr_category: bdiv_otu_table_subset_competitive.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                Df SumsOfSqs MeanSqs F.Model      R2  Pr(>F)  
qiime.data$map[[opts$category]] 23    11.243 0.48881   1.113 0.32989 0.01499 *
Residuals                       52    22.837 0.43917         0.67011          
Total                           75    34.079                 1.00000          
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
curr_cat: latitude
curr_metric: bray_curtis
curr_category: bdiv_otu_table_subset_competitive.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutati

In [11]:
#Boxplot summaries:
tissue_otu_table_dir = join(output_dir,"functional_groups")
n_permutations = 1000
categories = 'prop_Colony_maximum_diameter_universal,latitude,longitude'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/skbio/draw/_distributions.py:701: RuntimeWarning: Could not automatically resize plot to make room for axes labels and plot title. This can happen if the labels or title are extremely long and the plot size is too small. Your plot may have its labels and/or title cut-off. To fix this, try increasing the plot's size (in inches) and try again.
  RuntimeWarning)
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg rende

# Use adonis to explore how much variability in coral microbiome is explained by compartment

In [10]:
#Export Sceleractinian coral  specific biom file
subset_output = join(output_dir,'coral_only')
!filter_samples_from_otu_table.py -i $otu_table -o $subset_output/otu_table_coral_only.biom -m $mapping --sample_id_fp input/coral_only.txt --output_mapping_fp $subset_output/otu_table_coral_only_map.txt 

In [6]:
#Summarize BIOM to confirm filter functioned correctly
subset_output = join(output_dir,'coral_only')
!biom summarize-table -i $subset_output/otu_table_coral_only.biom -o $subset_output/otu_table_coral_only_summary.txt

#Summaries look good.

In [5]:
#Run beta diversity on rarified (1000) subset of each sample type
tissue_otu_table_dir = join(output_dir,"coral_only")
depth = 1000

for c in listdir(tissue_otu_table_dir):
    if not c.endswith(".biom"):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'bdiv_%s' %c)
    curr_otu_table = join(tissue_otu_table_dir,c)
    !beta_diversity_through_plots.py -i $curr_otu_table -o $curr_bdiv_dir -m $mapping -p $bc_prefs_fp -e $depth -f -t $tree_fp 

/macqiime/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.192225882455 and the largest is 17.2822267266.
  RuntimeWarning
/macqiime/anaconda/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the

In [9]:
#Check for differences among mapping categories within a given functional group
tissue_otu_table_dir = join(output_dir,"coral_only")

n_permutations = 1000
categories = 'BiologicalMatter'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)   
        for curr_cat in categories.split(","):
            print "curr_cat: %s" %curr_cat
            print "curr_metric: %s" %metric
            print "curr_category: %s" %c
            curr_compare_cat_output_dir = join(curr_bdiv_dir,"compare_categories_%s_%s" %(curr_cat,metric))
            !compare_categories.py -i $curr_dm -o $curr_compare_cat_output_dir -m $mapping -c $curr_cat --method adonis -n $n_permutations
            the_result = join(curr_compare_cat_output_dir,"adonis_results.txt")
            !head -n 30 $the_result

curr_cat: BiologicalMatter
curr_metric: bray_curtis
curr_category: bdiv_otu_table_coral_only.biom

Call:
adonis(formula = as.dist(qiime.data$distmat) ~ qiime.data$map[[opts$category]],      permutations = opts$num_permutations) 

Permutation: free
Number of permutations: 1000

Terms added sequentially (first to last)

                                 Df SumsOfSqs MeanSqs F.Model      R2   Pr(>F)
qiime.data$map[[opts$category]]   2    10.469  5.2344  11.443 0.03538 0.000999
Residuals                       624   285.434  0.4574         0.96462         
Total                           626   295.903                 1.00000         
                                   
qiime.data$map[[opts$category]] ***
Residuals                          
Total                              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
curr_cat: BiologicalMatter
curr_metric: weighted_unifrac
curr_category: bdiv_otu_table_coral_only.biom

Call:
adonis(formula = as.dist(qiime.data$distmat

In [ ]:
#Boxplot summaries:
tissue_otu_table_dir = join(output_dir,"coral_only")
n_permutations = 1000
categories = 'BiologicalMatter'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

# Explore differences in traunches of latitude (latitude_traunch) and prop_Colony_maximum_diameter_universal (prop_Colony_maximum_diameter_universal_traunch)

#Trauches added in "input/gcmp16S_map_r20_lat_size_traunches.txt"

In [5]:
#Boxplot summaries w/in all corals:
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"coral_only")
n_permutations = 1000
categories = 'latitude_traunch,prop_Colony_maximum_diameter_universal_traunch'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/skbio/draw/_distributions.py:701: RuntimeWarning: Could not automatically resize plot to make room for axes labels and plot title. This can happen if the labels or title are extremely long and the plot size is too small. Your plot may have its labels and/or title cut-off. To fix this, try increasing the plot's size (in inches) and try again.
  RuntimeWarning)
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg rende

In [6]:
#Boxplot summaries w/in tissue compartment:
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"tissue_compartment_tables")
n_permutations = 1000
categories = 'latitude_traunch,prop_Colony_maximum_diameter_universal_traunch'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/skbio/draw/_distributions.py:701: RuntimeWarning: Could not automatically resize plot to make room for axes labels and plot title. This can happen if the labels or title are extremely long and the plot size is too small. Your plot may have its labels and/or title cut-off. To fix this, try increasing the plot's size (in inches) and try again.
  RuntimeWarning)
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg rende

In [7]:
#Boxplot summaries w/in functional groups:
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"functional_groups")
n_permutations = 1000
categories = 'latitude_traunch,prop_Colony_maximum_diameter_universal_traunch'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/skbio/draw/_distributions.py:701: RuntimeWarning: Could not automatically resize plot to make room for axes labels and plot title. This can happen if the labels or title are extremely long and the plot size is too small. Your plot may have its labels and/or title cut-off. To fix this, try increasing the plot's size (in inches) and try again.
  RuntimeWarning)
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg rende

In [8]:
#Boxplot summaries w/in all corals for compartment differences:
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"coral_only")
n_permutations = 1000
categories = 'BiologicalMatter'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macq

In [9]:
#Boxplot summaries w/in brooders and spawners:
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"functional_groups")
n_permutations = 1000
categories = 'latitude_traunch,prop_Colony_maximum_diameter_universal_traunch'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_all_within --suppress_all_between -n $n_permutations --sort median

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/skbio/draw/_distributions.py:701: RuntimeWarning: Could not automatically resize plot to make room for axes labels and plot title. This can happen if the labels or title are extremely long and the plot size is too small. Your plot may have its labels and/or title cut-off. To fix this, try increasing the plot's size (in inches) and try again.
  RuntimeWarning)
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg rende

# Explore differences in traunches of prop_Colony_maximum_diameter_universal (< 20% vs > 20% max size) w/in compartments...

In [4]:
#Set file paths
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map_r20.txt")
output_dir = abspath("output/")

In [6]:
#Boxplot summaries w/in tissue compartment (with raw data save):
mapping = abspath("input/gcmp16S_map_r20_lat_size_traunches.txt")

tissue_otu_table_dir = join(output_dir,"tissue_compartment_tables")
n_permutations = 1000
categories = 'latitude_traunch,prop_Colony_maximum_diameter_universal_traunch'
for c in listdir(tissue_otu_table_dir):
    if not c.endswith('.biom'):
        continue
    curr_bdiv_dir = join(tissue_otu_table_dir,'%s' %c)
#    print listdir(curr_bdiv_dir)
#    curr_otu_table = join(tissue_otu_table_dir,c)
    for metric in ['bray_curtis','weighted_unifrac','unweighted_unifrac']:
        curr_dm = join(curr_bdiv_dir,"%s_dm.txt" %metric)
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_all_within_%s" %metric)
#        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_individual_between --suppress_individual_within  -n $n_permutations --sort median
        curr_boxplot_output_dir = join(curr_bdiv_dir,"distance_boxplots_%s" %metric)
        !make_distance_boxplots.py -d $curr_dm -o $curr_boxplot_output_dir -m $mapping -f $categories --height 20 --suppress_all_within --suppress_all_between -n $n_permutations --sort median --save_raw_data

/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macqiime/anaconda/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/macq

# With continuous data (i.e. prop_Colony_maximum_diameter_universal,latitude,longitude) it might be better to bin before box plots

# Could explore any interesting differences deeper by looking within compartments of specific functional groups.

In [4]:
#Summarize BIOM
!biom summarize-table -i $otu_table

Num samples: 692
Num observations: 129305
Total count: 9441738
Table density (fraction of non-zero values): 0.006

Counts/sample summary:
 Min: 2.0
 Max: 38523.0
 Median: 14010.000
 Mean: 13644.130
 Std. dev.: 7564.457
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
 E7.17.Pla.sine.1.20150622.M: 2.0
 E3.17.Ech.mamm.1.20150127.S: 3.0
 E10.17.Fav.pali.2.20150630.T: 5.0
 E7.17.Fav.spB.1.20150623.T: 7.0
 E3.17.Ech.horr.1.20150123.M: 8.0
 E3.15.Dip.heli.2.20150120.T: 9.0
 E10.Cyp.sera.1.20150628.M: 17.0
 E10.17.Fav.pali.1.20150630.T: 17.0
 E3.Out.Mil.sp.2.20150125.M: 24.0
 E10.17.Fav.pali.1.20150630.M: 36.0
 E10.17.Fav.pali.2.20150630.S: 41.0
 E10.Cyp.sera.2.20150628.T: 42.0
 E1.6.Acr.sp.1.20140726.T: 42.0
 E3.5.Gal.astr.2.20150127.T: 53.0
 E10.Ser.cali.2.20150627.T: 66.0
 E10.Alv.alli.1.20150628.S: 96.0
 E10.Ser.cali.1.20150627.T: 102.0
 E10.17.Fav.pali.2.20150630.M: 127.0
 E10.17.Fav.abdi.2.20150630.T: 165.0
 E7.17.Gon.edwa.2.20